In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utilities.OneHotEncoder import OneHotEncoder
from utilities.DataPreProcessor import DataPreProcessor

In [2]:
df_pdb_data_no_seq = pd.read_csv('resources/pdb_data_no_dups.csv')
df_pdb_seq = pd.read_csv('resources/pdb_data_seq.csv')
print("..Read data files..")

..Read data files..


In [3]:
# filtering out only protein structures
df_protein_seq = df_pdb_seq[(df_pdb_seq['macromoleculeType'] == 'Protein')]

In [4]:
# joining two data sets for protein Seq - classification

df_protein_final = df_protein_seq.merge(df_pdb_data_no_seq)

df_protein_seq_final = df_protein_final[['sequence', 'classification']]

df_protein_seq_final.head()

,sequence,classification
0,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,OXYGEN TRANSPORT
1,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAAKSE...,HYDROLASE(O-GLYCOSYL)
2,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,OXYGEN TRANSPORT
3,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNSLDAAK...,HYDROLASE(O-GLYCOSYL)
4,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,OXYGEN TRANSPORT


In [5]:
# drop na values in the data frame

df_protein_seq_final = df_protein_seq_final.dropna()
print("filtered data with NA values")

filtered data with NA values


In [6]:
##One_hot encoding

one_hot_encoder = OneHotEncoder()

one_hot_encoder.find_unique_amino_acids(df_protein_seq_final['sequence'])
print("Unique Amino Acid Set...")

Unique Amino Acid Set...


In [7]:
## creates an encoded dictionaryfor each Amino Acid
one_hot_encoder.create_one_hot_dict()

data_pre_processor = DataPreProcessor(one_hot_encoder,
                                      df_protein_seq_final['classification']
                                      .value_counts().to_dict())

In [8]:
## delete least preferred sequences
data_pre_processor.del_least_preferred()

df_protein_seq_final_for_modeling = \
    df_protein_seq_final[df_protein_seq_final['classification']
        .isin(data_pre_processor.count_dict_keys_as_list())]

df_protein_seq_final_for_modeling = \
    df_protein_seq_final_for_modeling.reset_index(drop=True)
print("final Shape of the DataSet {}".format(df_protein_seq_final_for_modeling.shape))

final Shape of the DataSet (291355, 2)


In [11]:
df_protein_seq_final_for_modeling = \
    df_protein_seq_final_for_modeling[df_protein_seq_final_for_modeling.apply(
        lambda x: len(x['sequence']) <=1200 and len(x['sequence']) >=50 ,axis=1
    )]

In [12]:
print("final Shape of the DataSet {}".format(df_protein_seq_final_for_modeling.shape))

final Shape of the DataSet (271914, 2)


In [9]:
X_sequences = df_protein_seq_final_for_modeling['sequence']
y_label = df_protein_seq_final_for_modeling['classification']

In [10]:
## test process sequences

data_pre_processor.process_all_seqs(X_sequences, y_label)
# print("Getting Processed Seqs")
# print(data_pre_processor.get_x())
# print("getting labels")
# print(data_pre_processor.get_y())
print("getting protein categories")
print(data_pre_processor.get_protein_categoeies())

getting protein categories
['ANTIMICROBIAL PROTEIN', 'APOPTOSIS', 'BIOSYNTHETIC PROTEIN', 'BLOOD CLOTTING', 'CELL ADHESION', 'CELL CYCLE', 'CHAPERONE', 'CONTRACTILE PROTEIN', 'CYTOKINE', 'DE NOVO PROTEIN', 'DNA BINDING PROTEIN', 'ELECTRON TRANSPORT', 'FLUORESCENT PROTEIN', 'GENE REGULATION', 'HORMONE', 'HORMONE/GROWTH FACTOR', 'HYDROLASE', 'HYDROLASE/HYDROLASE INHIBITOR', 'Hydrolase/Hydrolase Inhibitor', 'IMMUNE SYSTEM', 'ISOMERASE', 'LIGASE', 'LIPID BINDING PROTEIN', 'LYASE', 'LYASE/LYASE INHIBITOR', 'MEMBRANE PROTEIN', 'METAL BINDING PROTEIN', 'METAL TRANSPORT', 'MOTOR PROTEIN', 'OXIDOREDUCTASE', 'OXIDOREDUCTASE/ELECTRON TRANSPORT', 'OXIDOREDUCTASE/OXIDOREDUCTASE INHIBITOR', 'OXYGEN STORAGE/TRANSPORT', 'OXYGEN TRANSPORT', 'PEPTIDE BINDING PROTEIN', 'PHOTOSYNTHESIS', 'PLANT PROTEIN', 'PROTEIN BINDING', 'PROTEIN FIBRIL', 'PROTEIN TRANSPORT', 'REPLICATION', 'RNA BINDING PROTEIN', 'SIGNALING PROTEIN', 'STRUCTURAL GENOMICS, UNKNOWN FUNCTION', 'STRUCTURAL PROTEIN', 'SUGAR BINDING PROTEIN',

### LSTM Modeling

In [11]:
# from model.LSTMModel import LSTMModel

# lstm_model = LSTMModel(None,61)

In [12]:
# print(lstm_model.define_model(100,3))

In [13]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X_sequences, y_label, test_size=0.20, random_state=42)

In [14]:
# for X,y in data_pre_processor.batch_generator(512):
#     #print(y.shape)
#     lstm_model.get_model().fit(X,y,epochs=10,verbose=2)

In [15]:
from keras.layers import LSTM
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers import Dropout

Using TensorFlow backend.


In [16]:
aa_size = 25
num_features = 25

model = Sequential()
#model.add(Embedding(aa_size,100,input_length=None))
model.add(LSTM(100,input_shape=(None, num_features)))
model.add(Dropout(0.2))
model.add(Dense(len(data_pre_processor.get_protein_categoeies()),activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100)               50400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 61)                6161      
Total params: 56,561
Trainable params: 56,561
Non-trainable params: 0
_________________________________________________________________
None


In [17]:
for X,y in data_pre_processor.batch_generator(64):
    #print(X.shape)
    model.fit(X,y,epochs=2,verbose=2,validation_split=0.25)

Train on 48 samples, validate on 16 samples
Epoch 1/2
 - 6s - loss: 4.1096 - acc: 0.0625 - val_loss: 4.0986 - val_acc: 0.3125
Epoch 2/2
 - 1s - loss: 4.0998 - acc: 0.2708 - val_loss: 4.0891 - val_acc: 0.3125
Train on 48 samples, validate on 16 samples
Epoch 1/2
 - 1s - loss: 4.1008 - acc: 0.1667 - val_loss: 4.0682 - val_acc: 0.5000
Epoch 2/2
 - 1s - loss: 4.0896 - acc: 0.1667 - val_loss: 4.0424 - val_acc: 0.5000
Train on 48 samples, validate on 16 samples
Epoch 1/2
 - 2s - loss: 4.0747 - acc: 0.0417 - val_loss: 4.0772 - val_acc: 0.0000e+00
Epoch 2/2
 - 2s - loss: 4.0375 - acc: 0.0417 - val_loss: 4.0397 - val_acc: 0.0000e+00
Train on 48 samples, validate on 16 samples
Epoch 1/2
 - 2s - loss: 3.9676 - acc: 0.0000e+00 - val_loss: 4.0604 - val_acc: 0.1250
Epoch 2/2
 - 2s - loss: 3.5329 - acc: 0.0000e+00 - val_loss: 4.2018 - val_acc: 0.1250
Train on 48 samples, validate on 16 samples
Epoch 1/2
 - 1s - loss: 3.2575 - acc: 0.0417 - val_loss: 2.8699 - val_acc: 0.0000e+00
Epoch 2/2
 - 1s - loss

In [19]:
count = 0
for i in range(200,300,1):
    yhat = model.predict(np.array(data_pre_processor.get_x()[i])
                         .reshape(1,np.array(data_pre_processor.get_x()[i]).shape[0]
                                  ,np.array(data_pre_processor.get_x()[i]).shape[1]))
    
    act_y = np.array(data_pre_processor.get_y())[i]
    
    if np.argmax(yhat) == np.argmax(act_y):
        count += 1

print("Accuracy is {}".format(count))

Accuracy is 42
